In [523]:
import pandas as pd
import os
import gzip
import spacy as sp
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from collections import defaultdict
from sklearn.cluster import DBSCAN
from math import log
from nltk.corpus import wordnet as wn
import nltk
from itertools import product
import pickle
from matplotlib.mlab import frange
import warnings
warnings.filterwarnings('ignore')

In [44]:
nlp = sp.load('en')

In [498]:
#SCORE_WEIGHTS = [0.7, 0.05 ,0.289, 0.603, 10]
SCORE_WEIGHTS = [0.7, 0.7 ,0.289, 0.603, 10]
CANDIDATE_MOST_COMMON_NUMBER = 200
total_count = -1
MAX_CANDIDATES = 100

In [578]:
eval_data = pd.read_csv("eval-verb-noun.csv")

metaphor_words = []
metaphors = {}

for r in eval_data.iterrows():
    tokens = nlp(r[1]["Metaphor"].decode('utf-8', errors='ignore').strip().lower())
    topic = tokens[0]
    vehicle = tokens[-1]
    if topic.lemma_ not in metaphor_words:
        metaphor_words.append(topic.lemma_)
    if vehicle.lemma_ not in metaphor_words:
        metaphor_words.append(vehicle.lemma_)
    if r[1]["Metaphor"] not in metaphors:
        metaphors[r[1]["Metaphor"]] = [topic.lemma_, vehicle.lemma_, 
                                       [(str(r[1]["Interpretation"]).lower(), r[1]["Freq"])],
                                       [str(r[1]["Interpretation"]).lower()]
                                      ]
    else:
        metaphors[r[1]["Metaphor"]][2].append((str(r[1]["Interpretation"]).lower(), r[1]["Freq"]))
        metaphors[r[1]["Metaphor"]][3].append(str(r[1]["Interpretation"]).lower())
print metaphors

{'base career': [u'base', u'career', [('underpin', 5), ('found', 5), ('establish', 5)], ['underpin', 'found', 'establish']], 'limit significance': [u'limit', u'significance', [('inhibit', 5), ('hinder', 5), ('withhold', 5), ('restrict', 5), ('minimize', 5), ('reduce', 5)], ['inhibit', 'hinder', 'withhold', 'restrict', 'minimize', 'reduce']], 'waste time': [u'waste', u'time', [('misspend', 5), ('spend', 5)], ['misspend', 'spend']], 'dismiss contrast': [u'dismiss', u'contrast', [('discount', 5), ('ignore', 5), ('neglect', 5), ('discard', 5)], ['discount', 'ignore', 'neglect', 'discard']], 'glimpse duty': [u'glimpse', u'duty', [('recognize', 5), ('attend', 5), ('see', 5), ('understand', 5)], ['recognize', 'attend', 'see', 'understand']], 'accelerate change': [u'accelerate', u'change', [('quicken', 5), ('speed', 5)], ['quicken', 'speed']], 'overlook subject': [u'overlook', u'subject', [('ignore', 5), ('neglect', 5), ('miss', 5), ('dismiss', 5)], ['ignore', 'neglect', 'miss', 'dismiss']], '

In [162]:
class Lang:
    def __init__(self):
        self.vec = []
        self.word_count = 0
        self.ind2word = {}
        self.word2ind = {}
        for line in open("../glove.6B/glove.6B.300d.txt"):
            values = line.split(" ")
            v = []
            for i in range (1, len(values)):
                v.append(float(values[i]))
            self.vec.append(v)
            self.ind2word[self.word_count] = values[0]
            self.word2ind[values[0]] = self.word_count
            self.word_count += 1
    
    def get_vec(self, word):
        word = word.strip().lower()
        if word in self.word2ind:
            return self.vec[self.word2ind[word]]
        return None
            
lang = Lang()

In [609]:
candidates = pickle.load(open( "candidates-verb-noun.p", "rb" ) )

In [613]:
#print candidates['enthusiasm'].most_common(200)
print candidates

{'campaign': Counter({'is': 888, 'launched': 752, 'planning': 316, 'was': 287, 'has': 278, 'running': 262, 'run': 262, 'started': 255, 'began': 226, 'have': 194, "'s": 192, 'launch': 192, 'created': 190, 'ran': 188, 'called': 187, 'organized': 181, 'led': 181, 'support': 176, 'get': 175, 'be': 161, 'had': 156, 'start': 156, 'help': 148, 'create': 148, 'going': 144, 'make': 139, 'launching': 136, 'based': 136, 'end': 124, 'aimed': 113, 'bombing': 112, 'promote': 112, 'waged': 107, 'said': 103, 'made': 98, 'put': 97, 'stop': 96, 'announced': 96, 'join': 94, 'designed': 94, 'calling': 94, 'been': 92, 'continue': 90, 'do': 86, 'weaken': 85, 'focused': 84, 'including': 81, 'leading': 80, 'mounted': 79, 'joined': 78, 'raise': 77, 'bring': 74, 'take': 73, 'see': 73, 'says': 72, 'initiated': 72, 'set': 72, 'following': 72, 'done': 70, 'keep': 67, 'did': 66, 'aims': 66, 'took': 65, 'declared': 65, 'save': 62, 'kicked': 61, 'supported': 61, 'conducted': 59, 'failed': 59, 'continued': 59, 'waging

In [586]:
def calculate_sem_score(candidate, metaphor_component):
    #metaphor_component can be either the topic or the vehicle
    cand_vec = lang.get_vec(candidate)
    meta_comp_vec = lang.get_vec(metaphor_component)
    if meta_comp_vec is None:
        print metaphor_component, "vector doesn't exist"
        return -1
    if cand_vec is None:
        return -1
    return cosine_similarity(cand_vec, meta_comp_vec)

total_count = -1

def calculate_total_candidate_count():
    global total_count
    if total_count > -1:
        return total_count
    
    count = 0
    for key in candidates:
        for v in candidates[key]:
            count += candidates[key][v]
    total_count = count
    return count


candidate_counts_cache = {}

def candidate_count(candidate):
    if candidate in candidate_counts_cache:
        return candidate_counts_cache[candidate]
    
    count = 0
    for key in candidates:
        count += candidates[key][candidate]
    candidate_counts_cache[candidate] = count   #add to cache
    return count


def candidate_component_npmi(candidate, metaphor_component):
    if metaphor_component in candidates and candidate in candidates[metaphor_component]:
        c_x_y = candidates[metaphor_component][candidate]+1
    else:
        c_x_y = 1
    p_x_y = float(c_x_y) / float(calculate_total_candidate_count())
    c_x = candidate_count(candidate)
    p_x = float(c_x) / float(calculate_total_candidate_count())
    if metaphor_component in candidates:
        c_y = sum(candidates[metaphor_component].values())
    else:
        c_y = 1
    p_y = float(c_y) / float(calculate_total_candidate_count())
    pmi = log(p_x_y / (p_x * p_y))
    h_x_y = -log(p_x_y)
    npmi = pmi / float(h_x_y)
    return npmi
    

def calculate_final_score(candidate, topic, vehicle, weights):
    topic_sem_score = calculate_sem_score(candidate, topic)
    vehicle_sem_score = calculate_sem_score(candidate, vehicle)
    topic_npmi = candidate_component_npmi(candidate, topic)
    vehicle_npmi = candidate_component_npmi(candidate, vehicle)
    candidate_freq = candidate_count(candidate) / float(calculate_total_candidate_count())
    scores = [topic_sem_score, vehicle_sem_score, topic_npmi, vehicle_npmi, candidate_freq]
    #log linear score
    final_score = 0
    for i in range(0, 5):
        final_score += (weights[i] * (scores[i]))
    #return final_score, scores
    return final_score

In [603]:
def rank(topic, vehicle, candidate_most_common_number, 
         dbscan_radius=4, dbscan_samples=2, score_weights = SCORE_WEIGHTS, use_topic_candidates = True):
    if vehicle in candidates:
        most_vehicle_common = candidates[vehicle].most_common(candidate_most_common_number)
    else:
        most_vehicle_common = []
    
    if use_topic_candidates:
        if topic in candidates:
            most_topic_common = candidates[topic].most_common(candidate_most_common_number)
        else:
            most_topic_common = []
    else:
        most_topic_common = []
        
    scores = defaultdict(float)
    for c in most_vehicle_common:
        c = c[0]
        try:
            if len(wn.synsets(c, wn.ADJ)) > 0 or len(wn.synsets(c, wn.VERB)):
                scores[c] = calculate_final_score(c, topic, vehicle, score_weights)
        except UnicodeDecodeError, e:
            pass
    for c in most_topic_common:
        c = c[0]
        try:
            if len(wn.synsets(c, wn.ADJ)) > 0 or len(wn.synsets(c, wn.VERB)):
                scores[c] = calculate_final_score(c, topic, vehicle, score_weights)
        except UnicodeDecodeError, e:
            pass
    
    final_before_clustering = []
    rank_id = 1
    for c in sorted(scores, key=scores.get, reverse=True):
        #print rank_id, c, scores[c]
        rank_id = rank_id + 1
        final_before_clustering.append(c)
    
    #return final_before_clustering
    #clustering
    vectors = []
    final_before_clustering_filtered = []
    for x in final_before_clustering:
        v = lang.get_vec(x)
        if v is not None:
            vectors.append(v)
            final_before_clustering_filtered.append(x)
    if len(vectors) > 0:
        labels = DBSCAN(eps=dbscan_radius, min_samples=dbscan_samples).fit_predict(vectors)
    else:
        labels = []

    final_after_clustering = []
    clusters_seen_so_far = []
    clusters = {}
    for x in range(0, len(final_before_clustering_filtered)):
        candidate = final_before_clustering_filtered[x]
        label = labels[x]
        if label != -1 and label not in clusters_seen_so_far:
            clusters_seen_so_far.append(label)
            final_after_clustering.append(candidate)
            clusters[label] = [candidate]
        elif label == -1:
            final_after_clustering.append(candidate)
        else:
            clusters[label].append(candidate)
    #print clusters
    return final_after_clustering

def print_rank_results(res):
    for i in range(0, len(res)):
        print (i + 1), res[i]

In [604]:
#Vehicle + topic
#print_rank_results(rank("cigarette", "timebomb", CANDIDATE_MOST_COMMON_NUMBER, 4, 5, [0.6,1.1,0.1,0.1,3]))

#Vehicle only
print_rank_results(rank("limit", "significance", CANDIDATE_MOST_COMMON_NUMBER, 4, 2, [1.1,1.1,0.1,0.6,7], False))



In [611]:
def similar_words(wordx, wordy):
#     return wordx == wordy
    sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)
    for i,j in list(product(*[sem1,sem2])):
        if i == j:
            return True
    return False


def interpretation_equals(int1, int2):
    return similar_words(int1, int2)


def exists_in_interpretation_list(candidate, interpretations, min_freq):
    for i in interpretations:
        if i[1] >= min_freq and interpretation_equals(i[0], candidate):
            return i[0]
    return None


def get_valid_interpretation_number(interpretations, min_freq):
    c = 0
    for i in interpretations:
        if i[1] >= min_freq:
            c += 1
    return c


def interpretation_in(candidate, interpretations):
    for i in interpretations:
        if interpretation_equals(i, candidate):
            return True
    return False


def eval(dbscan_radius=4, dbscan_samples=2, score_weights = SCORE_WEIGHTS, use_topic_candidates = True):
    MRR = 0
    recall_at = {5:0.0, 10:0.0, 15:0.0, 25:0.0, 50:0.0}
    recall_at_participants = 0
    MAP = 0
    
    for m in metaphors:
        #print m, metaphors[m][0], metaphors[m][1]
        pred_interpretations = rank(metaphors[m][0], metaphors[m][1], CANDIDATE_MOST_COMMON_NUMBER, 
                                    dbscan_radius, dbscan_samples, score_weights, use_topic_candidates)
        MRR_added = False
        interpretations_seen_so_far = []
        valid_interpretation_number = get_valid_interpretation_number(metaphors[m][2], 5)
        if valid_interpretation_number > 0:
            recall_at_participants += 1
        relevants = 0
        curr_MAP = 0
        for i in range(0, min(MAX_CANDIDATES, len(pred_interpretations))):
            pi = pred_interpretations[i]
            #if pi in metaphors[m][3]:
            if interpretation_in(pi, metaphors[m][3]):
                if not MRR_added:
                    MRR += (1/float(i + 1))
                    MRR_added = True
                relevants += 1
                prec_at_k = float(relevants) / float(i + 1)
                curr_MAP += float(prec_at_k)
            if valid_interpretation_number > 0:
                inter = exists_in_interpretation_list(pi, metaphors[m][2], 5)
                if inter != None and inter not in interpretations_seen_so_far:
                    interpretations_seen_so_far.append(inter)
                if i in recall_at:
                    recall_at[i] += (float(len(interpretations_seen_so_far)) / float(valid_interpretation_number))
        if not MRR_added:
            MRR += 0
        if relevants > 0:
            curr_MAP /= float(relevants)
        MAP += curr_MAP
    MRR /= float(len(metaphors))
    MAP /= float(len(metaphors))
    results = []
    for i in sorted(recall_at):
        results.append(recall_at[i] / float(recall_at_participants))
    results.insert(0, MAP)    
    results.insert(0, MRR)
    return results


def eval_dbscan(use_topic_candidates = True):
    for r in range(1, 6):
        for s in range(1, 7):
            res = eval(r, s, use_topic_candidates = use_topic_candidates)
            res.insert(0, str(r) + "," + str(s))
            print('\t'.join([str(x) for x in res]))
            

def eval_weights(use_topic_candidates = True):
    for sem_top in frange(0.1, 1, 0.5):
        for sem_veh in frange(0.1, 1, 0.5):
            for pmi_top in frange(0.1, 1, 0.5):
                for pmi_veh in frange(0.1, 1, 0.5):
                    for freq in range(1, 20, 2):
                        res = eval(4, 1, [sem_top, sem_veh, pmi_top, pmi_veh, freq], use_topic_candidates = use_topic_candidates)
                        res.insert(0, str(sem_top) + "," + str(sem_veh) + "," + str(pmi_top) + "," + str(pmi_veh) + "," + str(freq))
                        print('\t'.join([str(x) for x in res]))
                        
def eval_results(dbscan_radius=4, dbscan_samples=2, score_weights = SCORE_WEIGHTS, use_topic_candidates = True):
    for m in metaphors:
        #print m, metaphors[m][0], metaphors[m][1]
        
        pred_interpretations = rank(metaphors[m][0], metaphors[m][1], CANDIDATE_MOST_COMMON_NUMBER, 
                                    dbscan_radius, dbscan_samples, score_weights, use_topic_candidates)
        MRR_added = False
        interpretations_seen_so_far = []
        valid_interpretation_number = get_valid_interpretation_number(metaphors[m][2], 5)
        relevants = 0
        curr_MAP = 0
        curr_MRR = 0
        recall_at = {5:0.0, 10:0.0, 15:0.0, 25:0.0, 50:0.0}
        for i in range(0, min(MAX_CANDIDATES, len(pred_interpretations))):
            pi = pred_interpretations[i]
            #if pi in metaphors[m][3]:
            if interpretation_in(pi, metaphors[m][3]):
                if not MRR_added:
                    curr_MRR = (1/float(i + 1))
                    MRR_added = True
                relevants += 1
                prec_at_k = float(relevants) / float(i + 1)
                curr_MAP += float(prec_at_k)
            if valid_interpretation_number > 0:
                inter = exists_in_interpretation_list(pi, metaphors[m][2], 5)
                if inter != None and inter not in interpretations_seen_so_far:
                    interpretations_seen_so_far.append(inter)
                if i in recall_at:
                    recall_at[i] += (float(len(interpretations_seen_so_far)) / float(valid_interpretation_number))

        if relevants > 0:
            curr_MAP /= float(relevants)
    
        results = []
        for i in sorted(recall_at):
            results.append(recall_at[i])
    
        results.insert(0, curr_MAP)    
        results.insert(0, curr_MRR)
        results.insert(0, m)
        print('\t'.join([str(x) for x in results]))

eval_results(4, 5, [0.6,1.1,0.1,0.1,3], False)

base career	0.0909090909091	0.0827654238822	0.0	0.333333333333	0.333333333333	0.333333333333	1.0
limit significance	0	0	0.0	0.0	0.0	0.0	0.0
waste time	0	0	0.0	0.0	0.0	0.0	0.0
dismiss contrast	0	0	0.0	0.0	0.0	0.0	0.0
glimpse duty	0	0	0.0	0.0	0.0	0.0	0.0
accelerate change	0	0	0.0	0.0	0.0	0.0	0.0
overlook subject	0	0	0.0	0.0	0.0	0.0	0.0
confront problem	0	0	0.0	0.0	0.0	0.0	0.0
stress activity	0	0	0.0	0.0	0.0	0.0	0.0
shape result	0	0	0.0	0.0	0.0	0.0	0.0
cast doubt	0.142857142857	0.156809757703	0.0	0.5	0.5	0.5	0.5
fix term	0	0	0.0	0.0	0.0	0.0	0.0
leak report	0.142857142857	0.163125001552	0.0	0.2	0.2	0.4	0.4
mend marriage	0.0714285714286	0.0577825936403	0.0	0.0	0.166666666667	0.166666666667	0.166666666667
follow profession	0	0	0.0	0.0	0.0	0.0	0.0
catch contagion	0	0	0.0	0.0	0.0	0.0	0.0
break agreement	0	0	0.0	0.0	0.0	0.0	0.0
stir excitement	1.0	0.390802581656	0.222222222222	0.333333333333	0.444444444444	0.555555555556	0.666666666667
reach agreement	0	0	0.0	0.0	0.0	0.0	0.0
impose control	0	0	

In [545]:
res = eval(4, 5, [0.6,1.1,0.1,0.1,3])
print('\t'.join([str(x) for x in res]))

0.312393807422	0.170370582125	0.198888888889	0.254444444444	0.278888888889	0.405555555556	0.562222222222
